# Chicago Food

## Milestone 2 : Preprossessing

In this milestone we will decribe what's inside our dataset. 

We will firstly make some cleaning process to be able to treat a coherous dataset for the following step. We will focus on four facilities type ; Restaurant, Grocery store, School and hospital. In the restaurant group, we will focus on the 5 biggest group company. We will enrich our database by adding a row with the area number of chicago. It will help us to do area analyse of our data.

From this clean dataset we will do a brute analysis on the risk, type of violation and number of inspection. We will contextualyse our data trought an analyse over the different area, the four facility type and the time.

Finaly, from the preprossing step we do, we will present what would be our analyse plan. 

#### Part 1. Cleaning the data

In [3]:
%matplotlib inline
import pandas as pd
import numpy as np
import re
from matplotlib.ticker import MaxNLocator
import matplotlib.pyplot as plt
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

In [4]:
#Loading the data
food = pd.read_csv('data/food-inspections.csv', sep=',')

In the cleaning process of the data we made the assumption that an inspection with no latitude and longitude information is not useful. The feature license should be different that 0 otherwise it's impossible to differenciate the type of facility inspection.

**Cleaning steps**

1) Drop column without any utility

2) Drop rows without latitude or longitude.

3) Drop the facilities having a license # equal to 0 

4) Convert the date format to an analysis friendly format

5) There are 3 risk levels (low, medium and high), any other value will be removed

6) Filter only the 4 facility types we are going to analyse

7) For restaurants, we are going to focus only the 5 most inspected chains of different types: McDonald's, Subway, Taco Bell, Satrbucks and Dunkin Donuts.

8) We will add spatial coordinates to the dataframe using 'Geopandas' and a map of districts found online.

**Step 1.**  Drop column without any utility

In [5]:
# Drop The columns whithout any links to our futher analyse 
# We will not use the zip code because some of them are wrong. We will only use latitude and longitude
# to localisate the facility 

food.drop(['Zip Codes','Historical Wards 2003-2015', 'Community Areas', 'Census Tracts','Wards','Zip','State','Address','Location'], axis=1, inplace=True)


**Step 2.** Drop row whitout latitude or longitude 

In [7]:
food.dropna(subset = ["Latitude", "Longitude"], inplace=True)

**Step 3.** Drop row with 0 as licence number.

In [8]:
food.drop(food[food['License #'] == 0].index, inplace=True)

**Step 4.** Convert the date format to an analysis friendly format

In [ ]:
# We only keep the year of the inspection
food["Inspection Date"] = food["Inspection Date"].str.split("-").str[0]

In [ ]:
#Check for missing years
food["Inspection Date"].isna().any()

**Step 5.** Drop risk different than high, medium, low

In [ ]:
#Checking for any other value than the 3 levels of risk
food.Risk.value_counts()

In [ ]:
#cleaning
food = food[~food["Risk"].str.contains('|'.join(["All"]))]
food.Risk.value_counts()

**Step 6.** Focus on the facility type: Restaurant, Grocery, School and Hospital

In [ ]:
# Check number of inpection per facility Type
food.groupby('Facility Type')['Facility Type'].count().sort_values(ascending=False).head(20)

In [ ]:
# We wil focus our analysis on the facility type: Restaurant, Grocery, School and Hospital.
food = food[food["Facility Type"].isin(["Restaurant","Grocery Store", "School", "Hospital"])]
food["Facility Type"].value_counts()

**Step 7.** For restaurant we focus only the 5 most inspected chains of different types of food 

In [ ]:
#Preview of restaurants
restaurants = food[food["Facility Type"]=="Restaurant"].copy()
#Drop NaN's
restaurants.dropna(subset=["AKA Name"], inplace=True)
#Only uppercase
restaurants["AKA Name"] = restaurants["AKA Name"].str.upper()

restaurants["AKA Name"].value_counts().head(25)

We creat a new datafram with only McDonald's, Subway, Taco Bell, Satrbucks and Dunkin Donuts.

**Step 8.** Add spatial coordinates to the dataframe

In [11]:
from requests import get
import json
import geopandas as gpd
from shapely.geometry import Point, Polygon
import folium

folium.__version__ == '0.10.0'

ModuleNotFoundError: No module named 'geopandas'

In [ ]:
url='https://data.cityofchicago.org/api/geospatial/cauq-8yn6?method=export&format=GeoJSON'
r = get(url)
geojson_data = r.json()
geojson = gpd.GeoDataFrame.from_features(geojson_data['features'])

geojson.head()

In [ ]:
geojson.drop(['perimeter', 'comarea_id','comarea'], axis=1, inplace=True)

In [ ]:
geometry = [Point(xy) for xy in zip(food['Longitude'], food['Latitude'])]
crs = {'init': 'epsg:4326'}
gdf = gpd.GeoDataFrame(food, crs=crs, geometry=geometry)

In [ ]:
food = gpd.sjoin(gdf, geojson, op='within', how='left')
food.reset_index(inplace=True, drop=True)
food.drop(['index_right'], axis=1, inplace=True)

In [ ]:
chicago_coord = [41.85, -87.7]
chicago_map = folium.Map(location=chicago_coord)

In [ ]:
area_area10 = food[food['area_num_1'] == '10'].copy()
area_area10.reset_index(drop=True, inplace=True)
#shape = area_zip25.shape

for index, row in area_area10.iterrows():
    if index < 30:
        folium.Marker([row["Latitude"], row["Longitude"]], popup=row['community'], 
                      icon=folium.Icon(color ='blue', icon = 'map-marker')).add_to(chicago_map)
    else: 
        break

In [ ]:
folium.GeoJson(geojson_data).add_to(chicago_map)

In [ ]:
chicago_map

#### Part 2. Brute Data analysis

#### Part 3. Context Data analysis

#### Part 4. What's next ?